## 학력, 교육연수, 혼인상태, 직업정보를 담은 연봉 데이터셋을 이용해 연봉 예측하기

### 1. 패키지 설치 및 데이터 로드

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb

file_url = 'https://media.githubusercontent.com/media/musthave-ML10/data_source/main/salary.csv'
data = pd.read_csv(file_url, skipinitialspace=True)
data.head()

,age,workclass,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
0,25,Private,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,NaN,Some-college,10,Never-married,NaN,Own-child,White,Female,0,0,30,United-States,<=50K


|변수|설명|
|---|--|
|age|연령|
|workclass|고용형태|
|education|학력|
|education_num|교육연수|
|marital-status|혼인상태|
|occupation|직업|
|relationship|가족관계|
|race|인종|
|sex|성별|
|captial_gain|자산증가|
|capital-loss|자산감소|
|hours-per-week|주당노동시간|
|native-country|본국|
|class|연봉구분|

In [5]:
# 종속변수 확인
data['class'].value_counts()

class
<=50K    37155
>50K     11687
Name: count, dtype: int64

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48842 entries, 0 to 48841
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             48842 non-null  int64 
 1   workclass       46043 non-null  object
 2   education       48842 non-null  object
 3   education-num   48842 non-null  int64 
 4   marital-status  48842 non-null  object
 5   occupation      46033 non-null  object
 6   relationship    48842 non-null  object
 7   race            48842 non-null  object
 8   sex             48842 non-null  object
 9   capital-gain    48842 non-null  int64 
 10  capital-loss    48842 non-null  int64 
 11  hours-per-week  48842 non-null  int64 
 12  native-country  47985 non-null  object
 13  class           48842 non-null  object
dtypes: int64(5), object(9)
memory usage: 5.2+ MB


In [7]:
# 연속형 변수만
data.describe()

,age,education-num,capital-gain,capital-loss,hours-per-week
count,48842.000000,48842.000000,48842.000000,48842.000000,48842.000000
mean,38.643585,10.078089,1079.067626,87.502314,40.422382
std,13.710510,2.570973,7452.019058,403.004552,12.391444
min,17.000000,1.000000,0.000000,0.000000,1.000000
25%,28.000000,9.000000,0.000000,0.000000,40.000000
50%,37.000000,10.000000,0.000000,0.000000,40.000000
75%,48.000000,12.000000,0.000000,0.000000,45.000000
max,90.000000,16.000000,99999.000000,4356.000000,99.000000


In [8]:
# 연속형 + 범주형 변수 모두
data.describe(include='all')

,age,workclass,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
count,48842.000000,46043,48842,48842.000000,48842,46033,48842,48842,48842,48842.000000,48842.000000,48842.000000,47985,48842
unique,NaN,8,16,NaN,7,14,6,5,2,NaN,NaN,NaN,41,2
top,NaN,Private,HS-grad,NaN,Married-civ-spouse,Prof-specialty,Husband,White,Male,NaN,NaN,NaN,United-States,<=50K
freq,NaN,33906,15784,NaN,22379,6172,19716,41762,32650,NaN,NaN,NaN,43832,37155
mean,38.643585,NaN,NaN,10.078089,NaN,NaN,NaN,NaN,NaN,1079.067626,87.502314,40.422382,NaN,NaN
std,13.710510,NaN,NaN,2.570973,NaN,NaN,NaN,NaN,NaN,7452.019058,403.004552,12.391444,NaN,NaN
min,17.000000,NaN,NaN,1.000000,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,1.000000,NaN,NaN
25%,28.000000,NaN,NaN,9.000000,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,40.000000,NaN,NaN
50%,37.000000,NaN,NaN,10.000000,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,40.000000,NaN,NaN
75%,48.000000,NaN,NaN,12.000000,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,45.000000,NaN,NaN


### 2. 전처리

#### (1) 범주형 변수

In [9]:
# 종속변수 class : <=50K 를 0, >50K를 1로 바꾸기
data['class'] = data['class'].map({'<=50K':0, '>50K':1})

In [13]:
# 범주형 변수만 모아보기
object_cols = list(data.select_dtypes('object').columns)

# 범주형 변수들의 고윳값 개수 확인
for obj in object_cols:
    print(obj, data[obj].nunique())

workclass 8
education 16
marital-status 7
occupation 14
relationship 6
race 5
sex 2
native-country 41


> 범주형 변수를 다룰 때는 더미변수로 바꾸어 활용하는 방법을 기본으로 생각하는 것이 좋다. 하지만 더미변수를 사용하면 값의 종류만큼 새로운 변수들이 생겨나기 때문에, 값의 종류가 수백 개면 그만큼 많은 변수가 생긴다. 필요하다면 수백개의 변수를 감수하여 모델링을 해야하지만, 꼭 필요하지 않다면 변수의 수를 줄일 방법을 강구할 필요가 있다.

> 값의 종류가 10개 미만인 변수는 그대로 두고, 10개 이상인 변수들만 확인하여 조치할 지를 검토. <br>
-> **education, occupation, native_country**

#### **education 범주형변수**

In [14]:
data['education'].value_counts()

education
HS-grad         15784
Some-college    10878
Bachelors        8025
Masters          2657
Assoc-voc        2061
11th             1812
Assoc-acdm       1601
10th             1389
7th-8th           955
Prof-school       834
9th               756
12th              657
Doctorate         594
5th-6th           509
1st-4th           247
Preschool          83
Name: count, dtype: int64

> 서열화가 가능하기 때문에 map 함수를 사용하여 숫자로 나타낼 수 있다. <br>


In [15]:
data['education-num'].value_counts()

education-num
9     15784
10    10878
13     8025
14     2657
11     2061
7      1812
12     1601
6      1389
4       955
15      834
5       756
8       657
16      594
3       509
2       247
1        83
Name: count, dtype: int64

> 하지만 education_num을 보니, education정보를 숫자로 표현한 것을 볼 수 있다.

In [17]:
# 확인해보기
np.sort(data['education-num'].unique())

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16],
      dtype=int64)

In [21]:
data[data['education'] == 'Preschool'].index

Index([  779,   818,  1029,  1059,  1489,  1498,  2364,  2465,  3037,  3540,
        4426,  4629,  4729,  5795,  7054,  7307,  7438,  7485,  7736,  7773,
       10304, 10721, 10777, 10954, 11456, 11677, 13568, 13582, 14153, 15513,
       15654, 15964, 16505, 17213, 19165, 19227, 19727, 19873, 20388, 22714,
       23145, 23351, 23454, 24175, 24361, 24369, 24377, 25056, 26591, 27415,
       27641, 28015, 29529, 31340, 32778, 32843, 34696, 36721, 37651, 37669,
       38003, 38075, 38448, 38812, 39221, 40456, 40839, 40979, 41381, 41394,
       41508, 41933, 42224, 42782, 42887, 43433, 43520, 44676, 48079, 48316,
       48505, 48640, 48713],
      dtype='int64')

In [20]:
data[data['education-num'] == 1].index

Index([  779,   818,  1029,  1059,  1489,  1498,  2364,  2465,  3037,  3540,
        4426,  4629,  4729,  5795,  7054,  7307,  7438,  7485,  7736,  7773,
       10304, 10721, 10777, 10954, 11456, 11677, 13568, 13582, 14153, 15513,
       15654, 15964, 16505, 17213, 19165, 19227, 19727, 19873, 20388, 22714,
       23145, 23351, 23454, 24175, 24361, 24369, 24377, 25056, 26591, 27415,
       27641, 28015, 29529, 31340, 32778, 32843, 34696, 36721, 37651, 37669,
       38003, 38075, 38448, 38812, 39221, 40456, 40839, 40979, 41381, 41394,
       41508, 41933, 42224, 42782, 42887, 43433, 43520, 44676, 48079, 48316,
       48505, 48640, 48713],
      dtype='int64')

> education을 숫자로 표현한 것이 education_num임을 확인

> 따라서 education 컬럼을 삭제하기로 결정

In [22]:
data.drop('education', axis=1, inplace=True)

#### **occupation 범주형 변수**

In [23]:
data['occupation'].value_counts()

occupation
Prof-specialty       6172
Craft-repair         6112
Exec-managerial      6086
Adm-clerical         5611
Sales                5504
Other-service        4923
Machine-op-inspct    3022
Transport-moving     2355
Handlers-cleaners    2072
Farming-fishing      1490
Tech-support         1446
Protective-serv       983
Priv-house-serv       242
Armed-Forces           15
Name: count, dtype: int64

> 이미 비슷한 직업군끼리는 묶인 상태로 정리된 것 같다. (ex. Farming-fishing) <br>
> 또한, 각 직업간의 서열이라고 할만한 부분도 없다. <br>
> 그대로 더미변수로 변환해야겠다.

#### **native_country 범주형 변수**

In [26]:
data['native-country'].value_counts()

native-country
United-States                 43832
Mexico                          951
Philippines                     295
Germany                         206
Puerto-Rico                     184
Canada                          182
El-Salvador                     155
India                           151
Cuba                            138
England                         127
China                           122
South                           115
Jamaica                         106
Italy                           105
Dominican-Republic              103
Japan                            92
Guatemala                        88
Poland                           87
Vietnam                          86
Columbia                         85
Haiti                            75
Portugal                         67
Taiwan                           65
Iran                             59
Greece                           49
Nicaragua                        49
Peru                             46
Ecuador      

> United_states가 압도적으로 큰 비중을 차지한다. 

> 방법1 : United_states / Not United_states(=others) 로 묶는 방법 
    - 데이터가 간소화되지만, 정보가 줄어드는 단점이 있다.

> 방법 2 : 지역별로 묶는 방법 (North America, South America, Asia)
    - 해당 지역에 속한 국가끼리 어느 정도 유사성(=종속변수의 유사성)을 보여야 무리가 없다. 
    - 즉, 국가별로 class에 대한 평균값을 내걸었을 때, 만약 Asia 국가들이 비슷한 수치를 보여주면 이렇게 묶는데 큰 무리가 없을 것이다.

In [29]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48842 entries, 0 to 48841
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             48842 non-null  int64 
 1   workclass       46043 non-null  object
 2   education-num   48842 non-null  int64 
 3   marital-status  48842 non-null  object
 4   occupation      46033 non-null  object
 5   relationship    48842 non-null  object
 6   race            48842 non-null  object
 7   sex             48842 non-null  object
 8   capital-gain    48842 non-null  int64 
 9   capital-loss    48842 non-null  int64 
 10  hours-per-week  48842 non-null  int64 
 11  native-country  47985 non-null  object
 12  class           48842 non-null  int64 
dtypes: int64(6), object(7)
memory usage: 4.8+ MB


In [39]:
num_cols = data.select_dtypes(np.number).columns
data[['native-country', 'age', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week', 'class']].groupby('native-country').mean().sort_values('class')

,age,education-num,capital-gain,capital-loss,hours-per-week,class
native-country,,,,,,
Holand-Netherlands,32.000000,10.000000,0.000000,2205.000000,40.000000,0.000000
Guatemala,32.090909,6.306818,167.875000,18.113636,38.715909,0.034091
Outlying-US(Guam-USVI-etc),38.826087,10.043478,0.000000,76.608696,41.347826,0.043478
Columbia,39.458824,9.258824,125.364706,65.247059,39.929412,0.047059
Dominican-Republic,37.970874,7.320388,1064.456311,39.029126,41.621359,0.048544
Mexico,33.635121,6.026288,415.954784,32.656151,40.213460,0.049422
Nicaragua,36.285714,9.000000,138.653061,69.938776,36.938776,0.061224
El-Salvador,33.380645,6.722581,392.761290,36.367742,36.361290,0.070968
Trinadad&Tobago,39.259259,8.962963,116.185185,156.518519,38.888889,0.074074


> United_States의 평균값은 0.243977 <br>
> 다른 국가들의 평균값은 United_states평균값보다 크거나 작아서 전혀 비슷하지 않음. <br>
> 즉, Others로 묶는 방법은 좋지 않아 보인다. 

> 그러면 지역별로 묶는 방법 고려. <br>
> 유럽 -> Frace : 0.421053 / Portugal : 0.179104 => 상당한 차이를 보인다. <br>
> 즉, 지역별로 묶는 방법도 좋지 않아 보인다.

> **범주형 데이터를 무작정 숫자로 치환하여 모델링하는 방법은 좋지 않다. 하지만 허용되는 경우가 있는데 바로 트리기반 모델을 사용할 때다.** 트리기반의 모델은 연속된 숫자들도 연속적으로 받아들이기보다 일정 구간을 나누어 받아들이기 때문에, 트리가 충분히 깊어지면 범주형 변수를 숫자로 바꾼다고 해도 큰 문제가 없다.

> **범주형데이터를 숫자로 치환하는 방법**
> 1. 랜덤하게 번호를 부여 (=단순한 라벨링) <br>
> 2. value_counts()함수로 확인한 숫자들을 부여 (하지만 중복된 숫자가 있는 경우엔 적합하지 않다.) <br> 
> 3. 평균값을 넣어주기 (하지만 오버피팅의 문제가 발생할 수 있다.)


> 현재 데이터에는 3번 평균값으로 치환하는 방법을 적용

In [42]:
# country 별로 class의 평균값 구하기
country_group=data[['native-country', 'age', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week', 'class']].groupby('native-country').mean()['class']

In [43]:
country_group = country_group.reset_index()
country_group

,native-country,class
0,Cambodia,0.321429
1,Canada,0.346154
2,China,0.295082
3,Columbia,0.047059
4,Cuba,0.246377
5,Dominican-Republic,0.048544
6,Ecuador,0.133333
7,El-Salvador,0.070968
8,England,0.370079
9,France,0.421053


In [44]:
# data와 country_group을 native_country 기준으로 결합(left join)
data = data.merge(country_group, on='native-country', how='left')
data

,age,workclass,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class_x,class_y
0,25,Private,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,0,0.243977
1,38,Private,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,0,0.243977
2,28,Local-gov,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,1,0.243977
3,44,Private,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,1,0.243977
4,18,NaN,10,Never-married,NaN,Own-child,White,Female,0,0,30,United-States,0,0.243977
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48837,27,Private,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,0,0.243977
48838,40,Private,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,1,0.243977
48839,58,Private,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,0,0.243977
48840,22,Private,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,0,0.243977


In [45]:
# class_y를 naive-country로 사용하고 + 기존 native-country는 삭제
data.drop('native-country', axis=1, inplace=True)
data = data.rename(columns={'class_x':'class', 'class_y':'native-country'})
data.head()

,age,workclass,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,class,native-country
0,25,Private,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,0,0.243977
1,38,Private,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,0,0.243977
2,28,Local-gov,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,1,0.243977
3,44,Private,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,1,0.243977
4,18,NaN,10,Never-married,NaN,Own-child,White,Female,0,0,30,0,0.243977


#### (2) 결측치 처리 및 더미 변수 변환

In [46]:
data.isna().mean() # 결측치의 비율 확인

age               0.000000
workclass         0.057307
education-num     0.000000
marital-status    0.000000
occupation        0.057512
relationship      0.000000
race              0.000000
sex               0.000000
capital-gain      0.000000
capital-loss      0.000000
hours-per-week    0.000000
class             0.000000
native-country    0.017546
dtype: float64

> 3개의 변수에서 결측치가 보인다. <br>
> workclass, occupation, native-country(class의 평균값으로 대체한 상태).<br> 
> median(), mean() 으로 결측치를 채우는 것도 가능.
> 임의의 숫자로 채우는 것도 가능(-9, -99)-> **트리기반모델에서는 큰 문제가 없지만, 선형모델에서는 데이터의 왜곡을 불러오니 주의해야한다.**

In [47]:
# native-country(연속형변수) 결측치는 임의의 숫자 -99 로 채우기로 결정
data['native-country'] = data['native-country'].fillna(-99)

# workclass, occupation(범주형변수)는 평균치로 해결할 수 없다. 이 경우 특정 텍스트로 채우거나 혹은 dropna()로 제거
print(data['workclass'].value_counts())
print(data['occupation'].value_counts())

workclass
Private             33906
Self-emp-not-inc     3862
Local-gov            3136
State-gov            1981
Self-emp-inc         1695
Federal-gov          1432
Without-pay            21
Never-worked           10
Name: count, dtype: int64
occupation
Prof-specialty       6172
Craft-repair         6112
Exec-managerial      6086
Adm-clerical         5611
Sales                5504
Other-service        4923
Machine-op-inspct    3022
Transport-moving     2355
Handlers-cleaners    2072
Farming-fishing      1490
Tech-support         1446
Protective-serv       983
Priv-house-serv       242
Armed-Forces           15
Name: count, dtype: int64


> workclass에서는 private비율이 압도적이다. 특정값이 대부분을 차지하는 경우라면 해당 값으로 결측치를 채워주는 방법도 무난하다. <br>
> occupation에서는 특정값이 압도적으로 많다고 하기 어렵다. 따라서 'unknown' 텍스트로 채우기로 결정. 

In [49]:
# workclass의 결측값은 private(특정값) 으로 채우기
data['workclass'] = data['workclass'].fillna('Private')

# occupation의 결측값은 unknown 텍스트로 채우기
data['occupation'] = data['occupation'].fillna('Unknown')

In [50]:
# 더미 변수 변환
data = pd.get_dummies(data,drop_first=True)

### 3. 학습, 테스트 셋 분리

In [51]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(data.drop('class', axis=1), data['class'], test_size=0.4, random_state=100)

print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(29305, 43) (29305,)
(19537, 43) (19537,)


### 4. 모델 학습

In [52]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

dt = DecisionTreeClassifier()
dt.fit(x_train, y_train)

pred = dt.predict(x_test)

print('의사결정나무 정확도:', accuracy_score(y_test, pred))

의사결정나무 정확도: 0.8155295081128116


### 5. GridSearchCV를 활용하여 최적의 하이퍼파라미터 찾기

In [54]:
from sklearn.model_selection import GridSearchCV

params = { 'criterion':['gini', 'entropy'],
          'max_depth': [5,10,15,20],
          'min_samples_split': [10,12,14,16]}

grid_cv = GridSearchCV(dt, param_grid = params, scoring='accuracy', cv=5, verbose=1)
grid_cv.fit(x_train, y_train)

print('GridSearchCV 최고 평균점수:', grid_cv.best_score_)
print('GridSearchCV 최적의 파라미터:', grid_cv.best_params_)

Fitting 5 folds for each of 32 candidates, totalling 160 fits
GridSearchCV 최고 평균점수: 0.8591025422282886
GridSearchCV 최적의 파라미터: {'criterion': 'gini', 'max_depth': 10, 'min_samples_split': 14}


In [56]:
result = pd.DataFrame(grid_cv.cv_results_)
result.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_depth,param_min_samples_split,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.037827,0.003894,0.003711,0.000727,gini,5,10,"{'criterion': 'gini', 'max_depth': 5, 'min_sam...",0.857362,0.851049,0.849855,0.853609,0.848661,0.852107,0.003096,19
1,0.032952,0.002037,0.003243,0.000695,gini,5,12,"{'criterion': 'gini', 'max_depth': 5, 'min_sam...",0.857192,0.851049,0.849855,0.853609,0.848661,0.852073,0.003039,20
2,0.031843,0.001357,0.002880,0.001290,gini,5,14,"{'criterion': 'gini', 'max_depth': 5, 'min_sam...",0.857362,0.851049,0.850196,0.853609,0.848661,0.852175,0.003049,17
3,0.034800,0.003375,0.001798,0.000980,gini,5,16,"{'criterion': 'gini', 'max_depth': 5, 'min_sam...",0.857362,0.851049,0.850196,0.853609,0.848661,0.852175,0.003049,17
4,0.060632,0.004167,0.002415,0.001283,gini,10,10,"{'criterion': 'gini', 'max_depth': 10, 'min_sa...",0.858898,0.859068,0.858215,0.861116,0.856680,0.858795,0.001434,4


In [60]:
result[['param_criterion', 'param_max_depth', 'param_min_samples_split', 'mean_test_score']].sort_values(by='mean_test_score')

,param_criterion,param_max_depth,param_min_samples_split,mean_test_score
12,gini,20,10,0.844805
13,gini,20,12,0.846579
14,gini,20,14,0.847432
15,gini,20,16,0.847671
28,entropy,20,10,0.850162
16,entropy,5,10,0.851015
17,entropy,5,12,0.851049
19,entropy,5,16,0.851118
18,entropy,5,14,0.851118
31,entropy,20,16,0.851220


> criterion='gini', max_depth=10, min_samples_split=14 일때, 테스트 점수가 가장 높은 것으로 확인